In [1]:
import requests, json, re, datetime

## вводные данные

Получение токена хорошо описано тут (полистайте вниз):
http://datalytics.ru/all/stanovyas-guru-api-yandeks-metriki/

Поля fields т.е. столбцы в отчете https://tech.yandex.ru/metrika/doc/api2/logs/fields/visits-docpage/

In [10]:
path = "D:/" # путь, куда будут сохраняться отчеты и логи ошибок
token = "AQAAAAAZnUV0AAOdM9W4wV44p0uQub-_7GzFV2Q"             # токен
m_id = "39797515"                                             # счетчик яндекс метрики
date1 = "2017-04-01"
date2 = "2017-04-01"
fields = "ym:s:clientID,ym:s:dateTime,ym:s:lastTrafficSource,ym:s:visitDuration,ym:s:pageViews"
source = "visits"

### Оценка возможности создания запроса
Справка: https://tech.yandex.ru/metrika/doc/api2/logs/queries/evaluate-docpage/

possible	Возможность создания запроса логов.

max_possible_day_quantity	Примерное максимально возможное количество дней с учетом текущей квоты.

In [3]:
#https://api-metrika.yandex.ru/management/v1/counter/{counterId}/logrequests/evaluate
    
url1 = "https://api-metrika.yandex.ru/management/v1/counter/"
# формируем ссылку. сцепляем вводные данные
evaluateURL = url1 + m_id +"/logrequests/evaluate?date1="+ date1 +"&date2="+ date2 +"&fields="+ fields +"&source="+ source +"&oauth_token="+ token
evaluateresult = requests.get(evaluateURL)                            # запрос к API
evaluateresult = evaluateresult.text.encode('utf-8').decode('utf-8')   # расшифровка ответа                                        
evaluateresult_dict = json.loads(evaluateresult)                       # превращаем ответ в словарь python, чтобы потом вытащить requestID
evaluateresult_dict

{'log_request_evaluation': {'max_possible_day_quantity': 39347,
  'possible': True}}

### Запрос на создание отчета
Справка https://tech.yandex.ru/metrika/doc/api2/logs/queries/createlogrequest-docpage/

In [4]:
url1 = "https://api-metrika.yandex.ru/management/v1/counter/"
# формируем ссылку. сцепляем вводные данные
postURL = url1 + m_id +"/logrequests?date1="+ date1 +"&date2="+ date2 +"&fields="+ fields +"&source="+ source +"&oauth_token="+ token
postresult = requests.post(postURL)                            # запрос к API
postresult = postresult.text.encode('utf-8').decode('utf-8')   # расшифровка ответа                                        
postresult_dict = json.loads(postresult)                       # превращаем ответ в словарь python, чтобы потом вытащить requestID
request_id = str(postresult_dict["log_request"]["request_id"]) # задаем переменную, в которую передаем request_id
postresult_dict

{'log_request': {'counter_id': 39797515,
  'date1': '2017-04-01',
  'date2': '2017-04-01',
  'fields': ['ym:s:clientID',
   'ym:s:dateTime',
   'ym:s:lastTrafficSource',
   'ym:s:visitDuration',
   'ym:s:pageViews'],
  'request_id': 82859,
  'source': 'visits',
  'status': 'created'}}

### Отмена создания отчета 
Если вы поняли, что послали формировать не тот отчет, лучше отменить его т.к есть ежедневные квоты на обращение к api могут закончиться.

In [ ]:
# формируем ссылку. сцепляем вводные данные
cancelURL = url1 + m_id +"/logrequest/"+ str(request_id) +"/cancel?oauth_token="+ token
cancelresult = requests.post(cancelURL)                            # запрос к API
cancelresult = cancelresult.text.encode('utf-8').decode('utf-8')   # расшифровка ответа                                        
cancelresult_dict = json.loads(cancelresult)
cancelresult_dict

### Статус готовности отчета
Справка: https://tech.yandex.ru/metrika/doc/api2/logs/queries/getlogrequest-docpage/

Значения статусов тут https://tech.yandex.ru/metrika/doc/api2/logs/queries/class_logrequest-docpage/#class_logrequest

In [8]:
# формируем ссылку. сцепляем вводные данные
getURL = url1 + m_id +"/logrequest/"+ request_id +"?&oauth_token="+ token
getresult = requests.get(getURL)                               # запрос к API
getresult = getresult.text.encode('utf-8').decode('utf-8')     # расшифровка ответа
getresult = json.loads(getresult)                       # превращаем ответ в словарь python, чтобы потом вытащить кол-во частей
getresult

{'log_request': {'counter_id': 39797515,
  'date1': '2017-04-01',
  'date2': '2017-04-01',
  'fields': ['ym:s:clientID',
   'ym:s:dateTime',
   'ym:s:lastTrafficSource',
   'ym:s:visitDuration',
   'ym:s:pageViews'],
  'parts': [{'part_number': 0, 'size': 308579}],
  'request_id': 82859,
  'size': 308579,
  'source': 'visits',
  'status': 'processed'}}

для продолжения требуется статус "processed"
### Скачивание отчета
Справка: https://tech.yandex.ru/metrika/doc/api2/logs/queries/download-docpage/

In [11]:
if getresult["log_request"]["status"] == "processed":
    len_parts = len(getresult["log_request"]["parts"])                       #Кол-во частей
    for i in range(len_parts):                                               #организуем цикл, чтобы записать все части в файл
        downloadURL = url1 + m_id +"/logrequest/"+ request_id +"/part/"+ str(i) +"/download?&oauth_token="+ token
        downloadresult = requests.get(downloadURL)                           # запрос к API
        downloadresult = downloadresult.text.encode('utf-8').decode('utf-8') # расшифровка ответа      
        namefile = "request_id " + request_id + ".tsv"                       # название файла, пример: "request_id 82744.tsv"
        # запись в файл
        with open(path + namefile, 'a', encoding='utf-8') as f:
            f.write(downloadresult)
    print('Отчет скачан и сохранен в папку, которую вы указали в начале файла в переменной "path"')
else: print('Отчет еще не сформирован')

Отчет скачан и сохранен в папку, которую вы указали в начале файла в переменной "path"


### Удаление отчета на сервере яндекс, который был скачан

In [12]:
# формируем ссылку. сцепляем вводные данные
cleanURL = url1 + m_id +"/logrequest/" + str(request_id) + "/clean?oauth_token="+ token 
clean = requests.post(cleanURL)                             # Отправляет запрос на удаление файлов 
json.loads(clean.text.encode('utf-8').decode('utf-8'))      # Ответ

{'log_request': {'counter_id': 39797515,
  'date1': '2017-04-01',
  'date2': '2017-04-01',
  'fields': ['ym:s:clientID',
   'ym:s:dateTime',
   'ym:s:lastTrafficSource',
   'ym:s:visitDuration',
   'ym:s:pageViews'],
  'parts': [{'part_number': 0, 'size': 308579}],
  'request_id': 82859,
  'size': 308579,
  'source': 'visits',
  'status': 'cleaned_by_user'}}